<a href="https://colab.research.google.com/github/KNU-BrainAI-Capstone2022/ITS/blob/main/review_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install wordcloud
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim

from transformers import BertTokenizer, BertForSequenceClassification

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


In [39]:
path = '/content/drive/MyDrive/fake reviews dataset.csv'

df = pd.read_csv(path)
print(df.shape)
df.head()

(40432, 4)


,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


In [40]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [41]:
labels = {'CG':0, # 가짜면 0
          'OR':1 # 진짜면 1
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['label']] # df 컬럼 잘 보고 설정
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 128, truncation=True,
                                return_tensors="pt") for text in df['text_']] # df 컬럼 잘 보고 설정

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [80]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(0*len(df)), int(0*len(df))]) # train: 70, validation: 20, test: 10

print(len(df_train),len(df_val), len(df_test))
print(len(df))

0 0 40432
40432


In [43]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased') # 'bert-base-cased': 영어용, 다른 언어의 데이터면 다른거 쓰기
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        # _: embedding vectors of all of the tokens in a sequence
        # pooled_output: embedding vector of [CLS] token, 분류 문제에선 이것만 써도 충분
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [44]:
# def saveModel(): 
#     path = '/content/drive/MyDrive/ClassifierModel.pth'
#     torch.save(model.state_dict(), path) 

In [46]:
# from tqdm import tqdm

# def train(model, train_data, val_data, learning_rate, epochs, cuda_switch):

#     train, val = Dataset(train_data), Dataset(val_data)

#     train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
#     val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

#     use_cuda = torch.cuda.is_available()
#     device = torch.device("cuda" if use_cuda else "cpu")

#     criterion = nn.CrossEntropyLoss()
#     optimizer = optim.Adam(model.parameters(), lr= learning_rate)

#     if cuda_switch:

#             model = model.cuda()
#             criterion = criterion.cuda()

#     best_accuracy = 0
#     for epoch_num in range(epochs):

#             total_acc_train = 0
#             total_loss_train = 0

#             for train_input, train_label in tqdm(train_dataloader):

#                 train_label = train_label.to(device)
#                 mask = train_input['attention_mask'].to(device)
#                 input_id = train_input['input_ids'].squeeze(1).to(device)

#                 output = model(input_id, mask)
                
#                 batch_loss = criterion(output, train_label)
#                 total_loss_train += batch_loss.item()
                
#                 acc = (output.argmax(dim=1) == train_label).sum().item()
#                 total_acc_train += acc

#                 model.zero_grad()
#                 batch_loss.backward()
#                 optimizer.step()
            
#             total_acc_val = 0
#             total_loss_val = 0

#             with torch.no_grad():

#                 for val_input, val_label in val_dataloader:

#                     val_label = val_label.to(device)
#                     mask = val_input['attention_mask'].to(device)
#                     input_id = val_input['input_ids'].squeeze(1).to(device)

#                     output = model(input_id, mask)

#                     batch_loss = criterion(output, val_label)
#                     total_loss_val += batch_loss.item()
                    
#                     acc = (output.argmax(dim=1) == val_label).sum().item()
#                     total_acc_val += acc
            
#             print(
#                 f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
#                 | Train Accuracy: {total_acc_train / len(train_data): .3f} \
#                 | Val Loss: {total_loss_val / len(val_data): .3f} \
#                 | Val Accuracy: {total_acc_val / len(val_data): .3f}')
#             accuracy= (total_acc_val / len(val_data))

#             if accuracy > best_accuracy: 
#                saveModel() 
#                best_accuracy = accuracy 
                  
# EPOCHS = 2 
model = BertClassifier()
# LR = 1e-6
              
# train(model, df_train, df_val, LR, EPOCHS, use_cuda)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [47]:
# def evaluate(model, test_data):

#     test = Dataset(test_data)

#     test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

#     use_cuda = torch.cuda.is_available()
#     device = torch.device("cuda" if use_cuda else "cpu")

#     if use_cuda:

#         model = model.cuda()

#     total_acc_test = 0
#     with torch.no_grad():

#         for test_input, test_label in test_dataloader:

#               test_label = test_label.to(device)
#               mask = test_input['attention_mask'].to(device)
#               input_id = test_input['input_ids'].squeeze(1).to(device)

#               output = model(input_id, mask)

#               acc = (output.argmax(dim=1) == test_label).sum().item()
#               total_acc_test += acc
    
#     print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
# evaluate(model, df_test)

In [81]:
result=[]
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:

        model = model.cuda()
    path = "/content/drive/MyDrive/ClassifierModel.pth"
    model.load_state_dict(torch.load(path))
    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)
              if(output.argmax(dim=1)[0]==1):
                result.append("OR")
              else:
                result.append("CG")
              
              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(result)
    print(len(test_data))
    print(total_acc_test)
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
 
evaluate(model, df_test)


['CG', 'CG', 'OR', 'CG', 'OR', 'OR', 'CG', 'OR', 'CG', 'OR', 'OR', 'CG', 'OR', 'OR', 'CG', 'CG', 'OR', 'CG', 'OR', 'CG', 'OR', 'OR', 'CG', 'CG', 'OR', 'OR', 'CG', 'CG', 'OR', 'OR', 'OR', 'CG', 'CG', 'CG', 'OR', 'OR', 'CG', 'OR', 'OR', 'CG', 'OR', 'CG', 'CG', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'CG', 'CG', 'OR', 'CG', 'CG', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'CG', 'CG', 'CG', 'CG', 'CG', 'OR', 'OR', 'CG', 'OR', 'OR', 'CG', 'OR', 'CG', 'OR', 'OR', 'OR', 'OR', 'CG', 'CG', 'CG', 'OR', 'OR', 'OR', 'CG', 'OR', 'CG', 'CG', 'OR', 'OR', 'OR', 'OR', 'CG', 'OR', 'OR', 'OR', 'CG', 'OR', 'CG', 'CG', 'CG', 'CG', 'OR', 'CG', 'CG', 'CG', 'CG', 'CG', 'CG', 'CG', 'OR', 'OR', 'CG', 'CG', 'OR', 'OR', 'OR', 'OR', 'CG', 'OR', 'CG', 'OR', 'OR', 'OR', 'OR', 'CG', 'CG', 'CG', 'OR', 'CG', 'CG', 'OR', 'CG', 'CG', 'CG', 'OR', 'OR', 'CG', 'OR', 'OR', 'CG', 'OR', 'CG', 'OR', 'CG', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'CG', 'OR', 'CG', 'CG', 'CG', 'CG', 'CG', 'CG', 'CG', 'CG', 'OR', 'OR

In [86]:
df['predicted_label'] = result
print(df.shape)
df.head()

(40432, 5)


,category,rating,label,text_,predicted_label
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor...",CG
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I...",CG
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...,OR
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i...",CG
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...,OR


In [90]:
df2 = df.loc[(df['predicted_label'] == 'OR') & (df['rating'] != 3) ]
print(df2.shape)

(18478, 5)


In [99]:
df2['division'] = np.select([df2.rating > 3], [1], default=0)
df2[50:55]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,category,rating,label,text_,predicted_label,division
95,Home_and_Kitchen_5,2.0,OR,These are so weak. They barely bring in any air.,OR,0
97,Home_and_Kitchen_5,4.0,OR,It feels cool to the touch but is super hard.,OR,1
98,Home_and_Kitchen_5,5.0,OR,"So cute, made well my grand daughter will love...",OR,1
99,Home_and_Kitchen_5,5.0,OR,Works as expected for connecting CCTV power su...,OR,1
101,Home_and_Kitchen_5,5.0,OR,Nice and contemporary. Great for the price. Ea...,OR,1


In [92]:
np.random.seed(112)
df2_train, df2_val, df2_test = np.split(df2.sample(frac=1, random_state=42), 
                                     [int(.7*len(df2)), int(.9*len(df2))]) # train: 70, validation: 20, test: 10

print(len(df2_train),len(df2_val), len(df2_test))

12934 3696 1848


In [93]:
def saveModel(): 
    path = '/content/drive/MyDrive/NetModel_s.pth'
    torch.save(model.state_dict(), path)

In [94]:
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs, cuda_switch):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)
    best_accuracy = 0
    
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr= learning_rate)

    if cuda_switch:

            model = model.cuda()
            criterion = criterion.cuda()
    

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0
            

            for train_input, train_division in tqdm(train_dataloader):

                train_division = train_division.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_division)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_division).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_division in val_dataloader:

                    val_division = val_division.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_division)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_division).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
            accuracy= (total_acc_val / len(val_data))

            if accuracy > best_accuracy: 
               saveModel() 
               best_accuracy = accuracy 

EPOCHS = 1 
model = BertClassifier()
LR = 1e-6
              
train(model, df2_train, df2_val, LR, EPOCHS, use_cuda)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 6467/6467 [08:53<00:00, 12.12it/s]


Epochs: 1 | Train Loss:  0.181                 | Train Accuracy:  0.827                 | Val Loss:  0.074                 | Val Accuracy:  0.949


In [95]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_division in test_dataloader:

              test_division = test_division.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_division).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
evaluate(model, df2_test)

Test Accuracy:  0.943


In [96]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    checking_num1 = 0
    checking_num2 = 0
    checking_num3 = 0
    checking_num4 = 0

    path = "/content/drive/MyDrive/NetModel_s.pth"
    model.load_state_dict(torch.load(path))
    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)
              if output.argmax(dim=1)[0] == 1:
                  checking_num1 +=1
              elif output.argmax(dim=1)[0] == 0:
                  checking_num3 +=1
              else:
                  checking_num4 +=1
              checking_num2+=1
              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    print('==',total_acc_test)
    print('==',len(test_data))
    print(checking_num1)
    print(checking_num2)
    print(checking_num3)
    print(checking_num4)
    print(f'Positive review : ', checking_num1)
    print(f'Total review : ', checking_num2)
    print(f'Positive review rate: {(checking_num1/checking_num2)*100: .3f}')
evaluate(model, df2_test)

Test Accuracy:  0.944
== 1744
== 1848
988
1848
860
0
Positive review :  988
Total review :  1848
Positive review rate:  53.463
